# 11 - Export validation CSV to create ground-truth

In [1]:
import pandas as pd
import glob
import json

In [2]:
import sys
import os
# Access to the utils directory
current_dir = os.getcwd()
utils_dir = os.path.join(current_dir, '..', 'utils')
sys.path.append(utils_dir)

In [3]:
from string_utils import TableValuesPostTreatment, NormalizeText

[nltk_data] Downloading package punkt to /home/STual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/STual/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
ROOT = "/home/STual/DAN-cadastre/inference/LHAY/"
RES = "/home/STual/DAN-cadastre/data/LHAY/ets_tab_p1"
JSONS = glob.glob(ROOT + '*.json')

## 1. Create CSVs

In [28]:
tabs = []
entities = ["Ⓒ","Ⓓ","Ⓔ","Ⓕ","Ⓖ"]
csv_columns = ["page_uuid","line_uuid","line_detection_order","line_true_order","line_status","taxpayer_num","taxpayer_cell","taxpayer_interpreted","name","firstnames","birthname","activities","addresses","titles","familystatus","plot_number","plot_address","plotnature","comment"]

for JSON in JSONS:
    with open(JSON,'r',encoding='utf8') as f:
        page = json.load(f)
    page_uuid = JSON.replace(ROOT,"").replace('.json','')

    tab = []
    counter = 0
    for line in page["entities"]:
        keys = list(line.keys())
        fkeys = keys[:-1]
        line_uuid = line["uri"]
        
        key = 'Ⓖ'
        if 'Ⓖ' in fkeys:#Numéro dans la table alphabétique
            taxpayer_num_cell = line[key]["text"]
        else:
            taxpayer_num_cell = ""

        key = "Ⓒ"
        if "Ⓒ" in fkeys: #Taxpayer
            taxpayer_cell = line[key]["text"]
            taxpayer_cell_i = line[key]["interpreted_text"] #A propager dans le tableau
            ner = line[key]["ner_interpreted_text_grouped"] #Si liste, séparer par des |. A propager dans le tableau si besoin
            name = ner["name"]
            firstnames = ner["firstnames"]
            birthname = ner["birthname"]
            if len(ner["activity"]) > 0:
                activities = "|".join(ner["activity"])
            else:
                activities = ""
            if len(ner["address"]) > 0:
                addresses = "|".join(ner["address"])
            else:
                addresses = ""
            if len(ner["title"]) > 0:
                titles = "|".join(ner["title"])
            else:
                titles = ""
            if len(ner["familystatus"]) > 0:
                familystatus = "|".join(ner["familystatus"])
            else:
                familystatus = ""
        else:
            taxpayer_cell, taxpayer_cell_i = "", ""
            
        key = "Ⓕ"
        if key in fkeys:
            plot_num = line[key]["text"]
        else:
            plot_num = ""
            
        key = "Ⓓ" 
        if key in fkeys:
            plot_address = line[key]["text"]
        else:
            plot_address = ""
            
        key = "Ⓔ"
        if key in fkeys:
            plot_nature = line[key]["text"]
        else:
            plot_nature = ""
            
        csv_line = [page_uuid,line_uuid,str(counter),"","",taxpayer_num_cell,taxpayer_cell,taxpayer_cell_i,name,firstnames,birthname,activities,addresses,titles,familystatus,plot_num,plot_address,plot_nature,""]
        assert len(csv_line) == len(csv_columns)
        tab.append(csv_line)
        counter+=1
    tabs.append(tab)

In [31]:
for tab in tabs:
    df = pd.DataFrame(tab,columns=csv_columns)
    page_uuid = df.iloc[0]["page_uuid"]
    df.to_csv(RES + "/" + page_uuid + '.csv', index=False) 

In [25]:
df

,page_uuid,line_uuid,line_detection_order,line_status,taxpayer_num,taxpayer_cell,taxpayer_interpreted,name,firstnames,birthname,activities,addresses,titles,familystatus,plot_number,plot_address,plotnature,comment
0,3ab13de2-f0b2-436f-9aa0-3b040e79975b,146f8c7a-b143-433f-b79e-89b398997c05,0,,26,Denoist (tanislas,Denoist (tanislas,Denoist (,tanislas,,,,,,359,la prairee,ee ervoir,
1,3ab13de2-f0b2-436f-9aa0-3b040e79975b,cec45d60-0594-4f21-9eee-0fb95467343b,1,,26,idem,Denoist (tanislas,Denoist (,tanislas,,,,,,360,d,pré,
2,3ab13de2-f0b2-436f-9aa0-3b040e79975b,565cb747-a932-442b-bdbb-3f27f1fc9cab,2,,7,Angot Jean baptiste,Angot Jean baptiste,Angot,Jean baptiste,,,,,,361,§,pré,
3,3ab13de2-f0b2-436f-9aa0-3b040e79975b,55d50b1e-1227-4038-8964-b74c74abff28,3,,26,Denoist ottanislas,Denoist ottanislas,Denoist,ottanislas,,,,,,362,§,pré,
4,3ab13de2-f0b2-436f-9aa0-3b040e79975b,e1e80c28-7afc-4154-98c7-6cc655dd7bad,4,,75,Chevalier loussaint,Chevalier loussaint,Chevalier,loussaint,,,,,,363,§,pré,
5,3ab13de2-f0b2-436f-9aa0-3b040e79975b,7cce6df7-42c0-494a-a12a-4800cf70f1bb,5,,75,idem,Chevalier loussaint,Chevalier,loussaint,,,,,,364,§,pné,
6,3ab13de2-f0b2-436f-9aa0-3b040e79975b,857999c1-d04e-4b87-9a3c-637599f974e7,6,,77,Chevzul anichel,Chevzul anichel,Chevzul,anichel,,,,,,365,§,Terre,
7,3ab13de2-f0b2-436f-9aa0-3b040e79975b,567fed6a-b030-4fd0-8485-81b86162fc10,7,,75,Chevalier loussannt,Chevalier loussannt,Chevalier,loussannt,,,,,,366,§,T,
8,3ab13de2-f0b2-436f-9aa0-3b040e79975b,17401ed5-8743-407a-925d-fbf2f5f17726,8,,72,Chevalier louis,Chevalier louis,Chevalier,louis,,,,,,367,§,T,
9,3ab13de2-f0b2-436f-9aa0-3b040e79975b,4b58a250-f4cd-45a8-86d5-429e43dea2b0,9,,94,Delaize aubourg↑o,Delaize aubourg↑o,Delaize aubourg,↑o,,,,,,368,§,Vigne,
